In [2]:
# Data: coaguluation time in seconds for blood drawn from 24 animals according to four 
# different diets
data = [[62,60,63,59], 
        [63,67,71,64,65,66], 
        [68, 66, 71, 67, 68, 68], 
        [56, 62, 60, 61, 63, 64, 63, 59]]

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from matplotlib import pylab
import pandas as pd
from pandas import DataFrame
import os

In [5]:
np.random.seed(123)

In [1]:
def theta_posterior(j , mi, sigma2, tau2, data):
    """ Evaluates the posterior distribution for the theta parameter of a group."""
    
    num = 1/tau2 * mi + len(data[j])/(sigma2)* np.mean(data[j])
    den = 1/tau2 + len(data[j])/sigma2
    thetajhat = num/den
    varhat = 1/den
    
    return stats.norm(thetajhat, np.sqrt(varhat))

In [2]:
help(theta_posterior)

Help on function theta_posterior in module __main__:

theta_posterior(j, mi, sigma2, tau2, data)
    Evaluates the posterior distribution for the theta parameters



In [7]:
def mi_posterior(theta, tau2):  
    """ Evaluates the posterior distribution for the mi parameter"""
    return stats.norm(np.mean(theta), np.sqrt(tau2/len(theta)))

In [8]:
def sigma2_posterior(theta, data):
    """ Evaluates the posterior distribution for the sigma^2 parameter"""
    
    n_vec = [len(group) for group in data]
    n = sum(n_vec)
    n_groups = len(data)
    a = [np.dot(data[i]-theta[i],data[i]-theta[i]) for i in range(n_groups)]
    sigma2hat = sum(a)/n
    return stats.invgamma(a = n / 2, scale = n * sigma2hat /2)

In [9]:
def tau2_posterior(theta, mi):
    """ Evaluates the posterior distribution for the tau^2 parameter"""
    
    J = len(theta)
    theta = np.array(theta)
    tau2hat = np.dot(theta - mi, theta - mi) / (J-1)
    return stats.invgamma(a = (J-1)/2, scale = (J-1)* tau2hat /2)

In [10]:
def set_starting_points(data, nchain):
    """ Sets the initial values for the theta parameter.
    
    The values are chosen by randomly chosing a value from the data.
    """
    
    starting_points = []
    for i in range(nchain):
        random_start = []
        for i in range(len(data)):
            random_start.append(np.random.choice(data[i]))        
        starting_points.append( random_start)
    return starting_points
    

In [14]:
def Gibbs_sampler(data, save = False, nchain = 10, iterations = 1000, dirname = 'Results'):
    """ Approximates the target distribution through the Gibbs sampler. 
    
    The underlying model assumes the presence of J groups whose distribution is assumed to be
    normal with same variance (sigma2_j = sigma2 for j=1..J) and different means (mi_j). 
    The mean parameters (mi_j for j=1..J) are assumed to follow a normal distribution with
    mean mi and variance tau2. 
    A uniform prior distribution is assumed for the joint distribution of 
    (mi, log(sigma), log(tau)).
        
    The function returns a dataframe obtained by concatenating the second half of the 
    values of each chain. Optionally the values for each chain are saved in csv files.
    """
    
    ngroups = len(data)
    starting_points = set_starting_points(data,nchain)
    colnames = ['theta' + str(i+1) for i in range(ngroups)] + ['mi', 'sigma', 'tau']
    
    mix_chain = DataFrame(columns = colnames)
           
    for iter_chain in range(nchain):
        
        theta = starting_points[iter_chain]
        chain = DataFrame(columns = colnames)
        
        mi = np.mean(theta)
        sigma2 = sigma2_posterior(theta, data).rvs(size=1)[0]
        tau2 = tau2_posterior(theta, mi).rvs(size=1)[0]
        
        chain.loc[0] = theta + [mi, np.sqrt(sigma2), np.sqrt(tau2)]
        
        for i in range(iterations-1):
    
            theta = [theta_posterior(j , mi, sigma2, tau2, data).rvs(size=1)[0] for j in range(len(theta))]    
            mi = mi_posterior(theta,tau2).rvs(size=1)[0]   
            tau2 = tau2_posterior(theta, mi).rvs(size=1)[0]
            sigma2 = sigma2_posterior(theta, data).rvs(size=1)[0]
            
            chain.loc[i+1] = theta + [mi, np.sqrt(sigma2), np.sqrt(tau2)]
            
        if save:
            current_dir = os.path.dirname('__file__')
            if not os.path.exists(os.path.join(current_dir,dirname)):
                os.makedirs(dirname)
            filename = os.path.join(current_dir, dirname, 'chain'+str(iter_chain+1)+'.csv')
            chain.to_csv(filename)
                   
        mix_chain = pd.concat([mix_chain, chain.tail(int(iterations/2))]).reset_index(drop=True)
            
    return mix_chain
           

In [12]:
def statistics(dataframe):
    """ Evaluates main quantiles for each column in the dataframe."""
    
    return dataframe.quantile([.05, .25, .5, .75, .95]).transpose()

In [16]:
result = Gibbs_sampler(data, save = True)

In [93]:
statistics(result)

,0.05,0.25,0.5,0.75,0.95
theta1,59.242838,60.432794,61.259861,62.079330,63.247126
theta2,64.254470,65.248407,65.914144,66.556151,67.532589
theta3,66.053438,67.122070,67.769473,68.448851,69.410380
theta4,59.738199,60.587317,61.156648,61.726947,62.592202
mi,57.688125,62.243204,63.934638,65.792899,70.093572
sigma,1.885053,2.165474,2.432425,2.703149,3.213782
tau,2.198704,3.495862,5.041563,7.955183,19.261650
